In [15]:
import glob
import os

In [16]:
#declare global variables
TRAIN_SET_DIR = '../archive/TRAIN_V2/data_out_2'
RANDOM_SEED = 42

In [19]:
#create array to store all the training song paths
song_paths = []
song_genres_nums = []

# loop over each music genre
for genre_num in os.listdir(TRAIN_SET_DIR):
    
    # in each music genre, add the paths of all the music to the array
    for song_path in os.listdir(os.path.join(TRAIN_SET_DIR, genre_num)):
        song_paths.append(song_path)
        song_genres_nums.append(genre_num)
                                
# check...

0--------------------------------------
0.wav
100.wav
10001.wav
10022.wav
10043.wav
10053.wav
10057.wav
10065.wav
10071.wav
10079.wav
10100.wav
10105.wav
10116.wav
10118.wav
10120.wav
10122.wav
10125.wav
10140.wav
10151.wav
10152.wav
10154.wav
10156.wav
10157.wav
10158.wav
10169.wav
10174.wav
10175.wav
10177.wav
10221.wav
1023.wav
10230.wav
10244.wav
1025.wav
10262.wav
10268.wav
10273.wav
10286.wav
10288.wav
10292.wav
10296.wav
10301.wav
10313.wav
10319.wav
10332.wav
10337.wav
10340.wav
10341.wav
10355.wav
10362.wav
10377.wav
10405.wav
10409.wav
10410.wav
10411.wav
10415.wav
10416.wav
10436.wav
10446.wav
10454.wav
10470.wav
1048.wav
10492.wav
10494.wav
10497.wav
1050.wav
10512.wav
10526.wav
10531.wav
10539.wav
10554.wav
10555.wav
1056.wav
10560.wav
10570.wav
10587.wav
10602.wav
10609.wav
1061.wav
10612.wav
10618.wav
10621.wav
10629.wav
1063.wav
10632.wav
1064.wav
10654.wav
1067.wav
10682.wav
10688.wav
10690.wav
10709.wav
10720.wav
10737.wav
10739.wav
10741.wav
10744.wav
10773.wav
10778